# SparkSQL Lab

In this lab, we will be working with data from [Libraries.io](http://Libraries.io), a package manager aggregator. Our data consist of two files, one detailing packages and package managers, the other detailing the code repositories the packages are developed in. 

In [1]:
data = spark.read.csv("hdfs:///data/projects-1.0.0-2017-06-15.csv",header=True, inferSchema=True, mode="DROPMALFORMED")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
19,application_1566055793802_0017,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
data = data.withColumn('SourceRank', data.SourceRank.cast("double"))
data = data.withColumn('Dependent Repositories Count', data["Dependent Repositories Count"].cast("double"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Make the data available to SQL using `createTempView`

In [3]:
data.createTempView('data')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The rest of the lab consists of answering questions about the data. 

### How many packages are accounted for in this dataset?

Hint: Use `count`

In [4]:
spark.sql("SELECT count(*) FROM data").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|count(1)|
+--------+
| 2211516|
+--------+

### What package managers are included in the data?

The package manager is in the column named 'platform'

Hint: `DISTINCT` can be used right in a `SELECT` statement

In [5]:
managers = spark.sql("SELECT DISTINCT platform from data")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

View the results using `collect`

In [6]:
managers.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(platform='NuGet'), Row(platform='Emacs'), Row(platform='Sublime'), Row(platform='Meteor'), Row(platform='CRAN'), Row(platform='Hex'), Row(platform='Pub'), Row(platform='Clojars'), Row(platform='Maven'), Row(platform='CocoaPods'), Row(platform='NPM'), Row(platform='Bower'), Row(platform='Packagist'), Row(platform='Homebrew'), Row(platform='Atom'), Row(platform='Elm'), Row(platform='Wordpress'), Row(platform='Julia'), Row(platform='SwiftPM'), Row(platform='Jam'), Row(platform='Pypi'), Row(platform='Inqlude'), Row(platform='Haxelib'), Row(platform='CPAN'), Row(platform='Nimble'), Row(platform='Shards'), Row(platform='PlatformIO'), Row(platform='Go'), Row(platform='Alcatraz'), Row(platform='Rubygems'), Row(platform='Dub'), Row(platform='Hackage'), Row(platform='Carthage'), Row(platform='Cargo')]

### What package has the higest SourceRank?

SourceRank is Libraries.io measure that combines popularity as well as how well maintained the packages is, along with a few other factors

Use `max` to find the largest SourceRank

In [7]:
spark.sql("SELECT max(SourceRank)from data").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|max(SourceRank)|
+---------------+
|           31.0|
+---------------+

Use a `WHERE` clause to select the row of the DataFrame that has the highest SourceRank

In [8]:
maxSR = spark.sql("SELECT * FROM data WHERE SourceRank = 31")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `show` to display the results 

In [9]:
maxSR.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------+----------+--------------------------------+---------------------+------------------+------------------------+----------+------+---------------------+----------------------------+-------------+
|     ID|Platform|               Name|   Created Timestamp|   Updated Timestamp|         Description|            Keywords|        Homepage URL|Licenses|      Repository URL|Versions Count|SourceRank|Latest Release Publish Timestamp|Latest Release Number|Package Manager ID|Dependent Projects Count|  Language|Status|Last synced Timestamp|Dependent Repositories Count|Repository ID|
+-------+--------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------+----------+--------------------------------+----

### What is the most frequent dependency per package manager?

To answer this question, let's break it down into smaller parts. 

Next use `groupBy` and `max` to find the highest number of "Dependent Repositories Count" per package manager

In [10]:
max_deps = spark.sql('SELECT platform, max(`Dependent Repositories Count`) FROM data GROUP BY platform')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `show` to look at this data

In [11]:
max_deps.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------------------------+
| platform|max(Dependent Repositories Count)|
+---------+---------------------------------+
|    NuGet|                          63559.0|
|    Emacs|                              0.0|
|  Sublime|                              0.0|
|   Meteor|                             72.0|
|     CRAN|                           6037.0|
|      Hex|                           5950.0|
|      Pub|                           2972.0|
|  Clojars|                          26779.0|
|    Maven|                          43397.0|
|CocoaPods|                          15781.0|
|      NPM|                         380978.0|
|    Bower|                         106457.0|
|Packagist|                         122137.0|
| Homebrew|                              0.0|
|     Atom|                              0.0|
|      Elm|                           4655.0|
|Wordpress|                              0.0|
|    Julia|                            540.0|
|  SwiftPM|                       

Notice how the column name includes the name of the aggregate function, in this case `max`. Re-write the query above using `AS` so this doesn't happen

In [12]:
max_deps = spark.sql('SELECT platform, max(`Dependent Repositories Count`) AS `Dependent Repositories Count` FROM data GROUP BY platform')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use the above query as a subquery, and filter out any rows that have a "Dependent Repositories Count" of 0

In [13]:
max_deps = spark.sql('SELECT platform,`Dependent Repositories Count` FROM (SELECT platform, max(`Dependent Repositories Count`) AS `Dependent Repositories Count` FROM data GROUP BY platform) WHERE `Dependent Repositories Count` != 0')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Check your work using `show`

In [14]:
max_deps.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------------------+
| platform|Dependent Repositories Count|
+---------+----------------------------+
|    NuGet|                     63559.0|
|   Meteor|                        72.0|
|     CRAN|                      6037.0|
|      Hex|                      5950.0|
|      Pub|                      2972.0|
|  Clojars|                     26779.0|
|    Maven|                     43397.0|
|CocoaPods|                     15781.0|
|      NPM|                    380978.0|
|    Bower|                    106457.0|
|Packagist|                    122137.0|
|      Elm|                      4655.0|
|    Julia|                       540.0|
|  SwiftPM|                       177.0|
|     Pypi|                     71110.0|
|     CPAN|                      2723.0|
|   Shards|                        66.0|
|       Go|                      5379.0|
| Rubygems|                    481616.0|
|      Dub|                       203.0|
+---------+----------------------------+
only showing top

Next, use `join` to join data with the above query (we've done this for you)

In [15]:
max_deps_info = max_deps = spark.sql("""
    SELECT data.platform, name, data.`Dependent Repositories Count` FROM data 
    JOIN (SELECT platform,`Dependent Repositories Count` FROM 
            (SELECT platform, max(`Dependent Repositories Count`) AS `Dependent Repositories Count` 
                    FROM data GROUP BY platform) WHERE `Dependent Repositories Count` != 0) AS X 
    ON X.`Dependent Repositories Count` == data.`Dependent Repositories Count` AND X.platform == data.platform
""") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Call `show` after to see the results. Pass a number to `show` so that all the results are shown.

In [16]:
max_deps_info.show(30)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+----------------------------+
| platform|                name|Dependent Repositories Count|
+---------+--------------------+----------------------------+
| Rubygems|                rake|                    481616.0|
|    NuGet|     Newtonsoft.Json|                     63559.0|
|       Go|golang.org/x/net/...|                      5379.0|
|      Dub|              vibe-d|                       203.0|
|      Elm|       elm-lang/core|                      4655.0|
|      Hex|              poison|                      5950.0|
|   Shards|               radix|                        66.0|
|CocoaPods|        AFNetworking|                     15781.0|
|    Bower|              jQuery|                    106457.0|
|Packagist|     phpunit/phpunit|                    122137.0|
|    Julia|              Compat|                       540.0|
|    Cargo|                libc|                      6922.0|
|     Pypi|            requests|                     71110.0|
| Cartha

### Who is the most proflific owner of packages per package manager?

For this next question, we need to consult the second file, which is detailed information about where and who develops each package. Reading in the data will be very similar to as was done above.

In [17]:
repos_data = spark.read.csv("hdfs:///data/repositories-1.0.0-2017-06-15.csv",header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
repos_data.createTempView("repos_data")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

To extract the owner into its own column, we are going to use the function `regexp_extract`

In [19]:
from pyspark.sql.functions import regexp_extract

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
repos_data = spark.sql("""SELECT *,regexp_extract(`Name With Owner`,'(.*)/',0) AS owner
                          FROM repos_data """)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Next, use `join` to create a joined DataFrame. The columns to join on are "ID" from the subquery above, and "Repository ID" from `data`. It will be helpful to use a subquery and the keyword `AS`

In [21]:
joined = spark.sql("""SELECT data.*, X.* FROM data JOIN (SELECT *,regexp_extract(`Name With Owner`,'(.*)/',0) AS owner 
                          FROM repos_data) AS X ON X.ID == data.`Repository ID`""")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Put this table back into the SQL catalog by using `createTempView`

In [22]:
joined.createTempView("joined")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now that we have the package owner information joined with the package, use `groupby` and `count` to see how many packages each owner has per package manager.

The relevant columns are "Platform" and "owner"

Hint: `groupBy` can take more than one column to group on. Use `AS` to give the aggregate function a good name

In [23]:
counts = spark.sql("SELECT platform, owner, count(*) from joined GROUP BY platform, owner")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Next we need to deterimine the max count per platform. Do this using `groupby` and `max`, using the previous query as a subquery

In [24]:
max_count = spark.sql("""
                        SELECT platform, max(count) FROM
                      (SELECT platform, owner, count(*) AS count from joined GROUP BY platform, owner)
                      GROUP BY platform
                      """)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Like before, we want to remove the name of the function from the column name, so re-write the query above to use `AS`

In [25]:
max_count = spark.sql("""
                        SELECT platform, max(count) AS count FROM
                      (SELECT platform, owner, count(*) AS count from joined GROUP BY platform, owner)
                      GROUP BY platform
                      """)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now the owner with the most packages per package manager can be found by using `join` on `counts` and `max_count`

Call `show` after joining to see the results. 

In [26]:
spark.sql("""SELECT A.platform, A.owner, A.count FROM
                (SELECT platform, owner, count(*) as count from joined GROUP BY platform, owner) AS A
                    JOIN
                       ( SELECT platform, max(count) AS count FROM
                      (SELECT platform, owner, count(*) AS count from joined GROUP BY platform, owner)
                      GROUP BY platform) AS B
                      on A.platform = B.platform AND A.count = B.count
                      """).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----------------+-----+
| platform|            owner|count|
+---------+-----------------+-----+
|    Cargo|        retep998/|  428|
|   Meteor|          rzymek/|  110|
|      Hex|  nerves-project/|   35|
|    NuGet|             aws/|  360|
| Homebrew|          google/|   19|
|    Julia|      JuliaStats/|   28|
|      Dub|     DerelictOrg/|   25|
|     CRAN|        ropensci/|  117|
|    Emacs|          syohex/|   45|
|   Shards|      ysbaddaden/|    7|
|      Elm|   elm-community/|   51|
|CocoaPods|       hyperoslo/|   50|
|CocoaPods|    nicklockwood/|   50|
|  Haxelib|      haxe-react/|   35|
|    Maven|        kiegroup/|  979|
|      Pub|       dart-lang/|  245|
|     Pypi|      collective/|  357|
| Rubygems|      jrobertson/|  301|
|Packagist|thecodingmachine/|  212|
|      Jam|        aureooms/|   56|
+---------+-----------------+-----+
only showing top 20 rows

### What is the correlation between number of github stars and number of times a package is listed as a dependency?

Call `corr` on the table, passing it the correct column names

In [27]:
spark.sql("SELECT corr(`Stars Count`, `Dependent Repositories Count`) FROM joined").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------+
|corr(CAST(Stars Count AS DOUBLE), Dependent Repositories Count)|
+---------------------------------------------------------------+
|                                            0.06641716677190267|
+---------------------------------------------------------------+

### Which package names are found in both npm and pypi
For the final question, we are going to use set operations.

First we need to find the packages in pypi and the packages in NPM

Use `WHERE` to find all the elements of the DataFrame whose "Platform" is equal to "Pypi", and `SELECT` to only return the names

In [28]:
pypi = spark.sql("SELECT name FROM data WHERE platform = 'Pypi'")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `WHERE` to find all the elements of the DataFrame whose "Platform" is equal to "NPM", and `SELECT` to only return the names

In [29]:
npm = spark.sql("SELECT name FROM data WHERE platform = 'NPM'")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use `INTERSECT` to get the names that appear in both

In [30]:
intersection = spark.sql("SELECT name FROM data WHERE platform = 'Pypi'INTERSECT SELECT name FROM data WHERE platform = 'NPM'")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

View the names that appear in both by calling `show`

In [31]:
intersection.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|                name|
+--------------------+
|               anime|
|           arguments|
|             bitlist|
|           bookshelf|
|                 bsw|
|           carbonate|
|                clog|
|cloudfront-log-pa...|
|            collectd|
|               crest|
|                 dbt|
|                earl|
|                elsa|
|                foxy|
|                 gir|
|                guts|
|            habanero|
|              honcho|
|                hope|
|                 hud|
+--------------------+
only showing top 20 rows